# Utilities

Set dataset path and scale applied to spines during visualization.

In [ ]:
from notebook_widgets import SpineMeshDataset

# set dataset path and scale applied to spines during visualization
dataset_path = "example_dendrite"
scale = (1, 1, 1)

# load and scale spine mesh dataset
spine_dataset = SpineMeshDataset().load(dataset_path)
scaled_spine_dataset = SpineMeshDataset().load(dataset_path)
scaled_spine_dataset.apply_scale(scale)

### View Dendrite Skeleton

In [ ]:
from notebook_widgets import view_skeleton_widget


display(view_skeleton_widget(scaled_spine_dataset))

### View Dendrite segmentation

In [ ]:
from notebook_widgets import dendrite_segmentation_view_widget

display(dendrite_segmentation_view_widget(scaled_spine_dataset))

### View Chords
Set `num_of_chords` and `num_of_bins`. Histograms can be exported to `dataset_path/chords_%num_of_chords%_chords_%num_of_bins%_bins.csv` file. Histograms are only exported for meshes that were viewed! Otherwise it would take too much time to calculate. Use `Calculate Metrics` cell to calculate metrics for the entire dataset.

In [ ]:
from notebook_widgets import spine_chords_widget


num_of_chords = 30000
num_of_bins = 100

display(spine_chords_widget(spine_dataset, scaled_spine_dataset, dataset_path, num_of_chords, num_of_bins))

### Calculate Metrics
Metrics for the dataset will be saved to `dataset_path/metrics.csv`. Chords histograms will be saved separately to `dataset_path/chords.csv`. 

In [ ]:
from spine_metrics import SpineMetricDataset


# chord method parameters
num_of_chords = 30000
num_of_bins = 100

# calculate metrics
metric_names = ["OldChordDistribution", "OpenAngle", "CVD", "AverageDistance",
                "LengthVolumeRatio", "LengthAreaRatio", "JunctionArea",
                "Length", "Area", "Volume", "ConvexHullVolume", "ConvexHullRatio"]
metric_params = [{"num_of_chords": num_of_chords, "num_of_bins": num_of_bins},
                 {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}]
spine_metrics = SpineMetricDataset()
spine_metrics.calculate_metrics(spine_dataset.spine_meshes, metric_names, metric_params)
spine_metrics.save(f"{dataset_path}/metrics.csv")    

spine_metrics.get_metrics_subset(["OldChordDistribution"]).save_as_array(f"{dataset_path}/chords.csv")

### View Spines in Dataset

In [ ]:
from spine_metrics import SpineMetricDataset
from notebook_widgets import spine_dataset_view_widget


metrics = SpineMetricDataset().load(f"{dataset_path}/metrics.csv")
display(spine_dataset_view_widget(scaled_spine_dataset, metrics))

### Merge Manual Classifications

In [ ]:
from spine_metrics import SpineMetricDataset
from notebook_widgets import create_dir, remove_file
from spine_fitter import SpineGrouping
import pathlib

dataset_path = "0.025 0.025 0.1 dataset"
merged_path = f"{dataset_path}/manual_classification/manual_classification_merged.json"
merged_reduced_path = f"{dataset_path}/manual_classification/manual_classification_merged_reduced.json"

# remove old merged classifications
remove_file(merged_path)
remove_file(merged_reduced_path)

# load manual classifications
path = pathlib.Path(f"{dataset_path}/manual_classification")
classification_paths = [str(classification_path) for classification_path in path.glob("*.json")]
groupings = [SpineGrouping().load(path) for path in classification_paths]
print(f"Merging manual classifications: {classification_paths}\n")

# merge classifications
merged_grouping = SpineGrouping.merge(groupings, outliers_label="Unclassified")
merged_grouping.save(merged_path)
print(f"Saved to \"{merged_path}\".")

# remove spines with no consensus, filopodias and labeled-as-outliers
contested = SpineGrouping.get_contested_samples(groupings)
merged_grouping.remove_samples(contested)
merged_grouping.remove_samples(merged_grouping.groups["Filopodia"].copy())
merged_grouping.remove_samples(merged_grouping.groups["Outlier"].copy())
del merged_grouping.groups["Filopodia"]
del merged_grouping.groups["Outlier"]

merged_grouping.save(merged_reduced_path)
print(f"Saved to \"{merged_reduced_path}\".")

### Consensus Table

In [ ]:
from notebook_widgets import consensus_widget
from spine_fitter import SpineGrouping
import pathlib

# load manual classifications
path = pathlib.Path(f"{dataset_path}/manual_classification")
classification_paths = path.glob("manual_classification_?.json")
groupings = [SpineGrouping().load(str(path)) for path in classification_paths]

# show consensus table
display(consensus_widget(groupings))

### Merge Train & Test Manual Classifications

In [ ]:
from spine_fitter import SpineGrouping
import pathlib


def remove_prefix(grouping: SpineGrouping, prefix: str) -> None:
    for label, group in grouping.groups.items():
        new_group = set()
        for spine_name in group:
            new_group.add(spine_name.replace(prefix, ""))
        grouping.groups[label] = new_group

    new_samples = set()    
    for spine_name in grouping.samples:
        new_samples.add(spine_name.replace(prefix, ""))
    grouping.samples = new_samples


path = pathlib.Path(f"test 0.025 0.025 0.1 dataset/manual_classification")
classification_paths = path.glob("manual_classification_?.json")

for classification_path in classification_paths:
    grouping_test = SpineGrouping().load(str(classification_path))
    grouping_train = SpineGrouping().load(f"train 0.025 0.025 0.1 dataset/manual_classification/{classification_path.name}")

    remove_prefix(grouping_train, "train ")
    remove_prefix(grouping_test, "test ")

    merged = SpineGrouping.merge([grouping_train, grouping_test])
    merged.save(f"0.025 0.025 0.1 dataset/manual_classification/{classification_path.name}")
    
    print(f'Saved classification to "0.025 0.025 0.1 dataset/manual_classification/{classification_path.name}"')